In [1]:
# ingest Web of Science csv files into mySQL database

In [2]:
import sqlalchemy
import pandas as pd
#import mysql.connector as mysql
import pymysql
import time
import os

master_timer_start = time.time()

In [3]:
#      specifies how to recognize the partial file splits:

data_dir = '/project2/jevans/study_dbs/data/full_v1/'

pubs_substring = 'wos_cut_publications'  # postfixed with e.g. batch_0.csv
contrib_substring = 'wos_cut_contributors'
inst_substring = 'wos_cut_institutions'
ref_path = 'wos_cut_refs.csv'
wosIDs_path = 'wos_cut_wosids.csv'

# sql alchemy connection string:
db_name = 'test_wos_cut_full'
connect_string = "mysql+pymysql:///{}?unix_socket=/project2/jevans/study_dbs/mysql/.sql.sock".format(db_name)


In [4]:
# create the database

client_config = {'unix_socket':'/project2/jevans/study_dbs/mysql/.sql.sock'}
db = pymysql.connect(**client_config)



cursor = db.cursor()
sql = "CREATE DATABASE {}".format(db_name)
try:
    cursor.execute(sql)
except Exception as e:
    print(e)
cursor.close()
db.close()

In [5]:
# build a dictionary for to keep track of file names, etc:

tables = {}
tables['publications'] = {'substring_filemarker':pubs_substring}
tables['contributors'] = {'substring_filemarker':contrib_substring}
tables['institutions'] = {'substring_filemarker':inst_substring}
tables['the_references'] = {'substring_filemarker':ref_path}
tables['wosIDs'] = {'substring_filemarker':wosIDs_path}

In [6]:
# sort out the matching file splits

In [7]:
all_datafiles = os.listdir(data_dir)

for k, v in iter(tables.items()):
    
    print('processing {} ...'.format(k))
    start_time = time.time()
    
    tables[k]['files'] = [f for f in all_datafiles \
                          if (v['substring_filemarker'] in f)]
    DFs = []
    for filename in tables[k]['files']:
        path = os.path.join(data_dir,filename)
        #print(path)
        DFs.append(pd.read_csv(path, sep=','))

    # join dfs
    df = pd.concat(DFs, axis=0, ignore_index=True)        

    # insert into db
    sql_engine = sqlalchemy.create_engine(connect_string)
    df.to_sql(con=sql_engine,
             name=k,
             if_exists='replace',
             chunksize=50000)
        
    end_time = time.time()
    print('{} ingested in {} s'.format(k, end_time-start_time))


processing publications ...
publications ingested in 575.8294894695282 s
processing contributors ...
contributors ingested in 465.612087726593 s
processing institutions ...
institutions ingested in 18.747880697250366 s
processing the_references ...
the_references ingested in 1522.6044552326202 s
processing wosIDs ...
wosIDs ingested in 87.0508086681366 s


In [8]:
master_timer_stop = time.time()
print('total elapsed time during ingestion: {} s'.format(master_timer_stop - master_timer_start))

total elapsed time during ingestion: 2669.903683900833 s
